In [1]:
SCHOOL="brown"
SUBJECT="opinions"
START_YEAR=2010
FINAL_YEAR=2022
DATA_DIR="data"
OUTPUT_DIR="output"

In [2]:
%pwd

'/Users/angela/diversity-colleges/notebooks'

In [3]:
cd ..

/Users/angela/diversity-colleges


In [4]:
import sys
sys.path.append('src')
import ouraws
import ourgraphs
import textutil

In [5]:
S3OBJECT_KEY = f"{DATA_DIR}/{SCHOOL}-{SUBJECT}-SNAPSHOT.parquet"
df = ouraws.getFromS3(S3OBJECT_KEY)

In [6]:
df.shape

(4474, 4)

In [7]:
df.head(2)

,url,body,year,month
0,https://www.browndailyherald.com/article/2023/...,\nAs a self-proclaimed “Abbott Elementary” fan...,2023,2
1,https://www.browndailyherald.com/article/2023/...,"\nWhen I was eight, my sister bought me a note...",2023,2


In [8]:
import requests
import os

BIPARTISAN_API_KEY = os.environ.get("BIPARTISAN_API")

BIPARTISAN_URL = "https://api.thebipartisanpress.com/api/endpoints/beta/robert"

In [9]:
df2022 = df[df['year'] == 2022]

articles_list = df2022['body'].to_list()
for article in articles_list:
    payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
    response = requests.post(BIPARTISAN_URL, data=payload)
    print(response.text)
    break

-6.556617


In [13]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013,
       2012, 2011, 2010, 2009])

In [14]:
import pandas as pd

RESULTS_FILE = f"{DATA_DIR}/{SCHOOL}-POLARITY.parquet"

In [16]:
import asyncio

results = []

for year in range(2009, 2023):
    df2 = df[df['year'] == year]
    print(f"Year: {year} ... {df2.shape[0]}")

    articles_list = df2['body'].to_list()
    value_sum = 0.0
    article_count = 0
    for article in articles_list:
        payload = {"API": BIPARTISAN_API_KEY, "Text": article.encode("utf-8")}
        response = requests.post(BIPARTISAN_URL, data=payload)
        try:
            value_sum += float(response.text)
            article_count += 1
            # print(f"{article_count}", end=".")
        except:
            pass  # this is a non-number
        
    results.append({
        'year' : year,
        'article_count' : article_count,
        'polarity_sum' : value_sum,
        'polarity_avg' : float(value_sum) / article_count
    })

    print(f"{year}\t{article_count}\t{value_sum}\t{value_sum/article_count}")
# for each year, get all articles' bodies & send to bipartisan press API 
#    and sum up the values & output the average for each year

Year: 2009 ... 281
2009	281	-715.736284	-2.5471042135231317
Year: 2010 ... 427
2010	427	-1277.3148360000018	-2.9913696393442666
Year: 2011 ... 422
2011	422	-1549.3093449999994	-3.671349158767771
Year: 2012 ... 436
2012	436	-1460.6400310000004	-3.3500918142201845
Year: 2013 ... 399
2013	399	-1629.6559999999995	-4.084350877192981
Year: 2014 ... 384
2014	384	-1772.3946520000002	-4.615611072916667
Year: 2015 ... 345
2015	345	-1790.590606	-5.190117698550725
Year: 2016 ... 315
2016	315	-1898.4783969999992	-6.026915546031743
Year: 2017 ... 298
2017	298	-2505.8366180000007	-8.408847711409399
Year: 2018 ... 309
2018	309	-2395.7654019999986	-7.753286090614882
Year: 2019 ... 278
2019	278	-1955.7454199999997	-7.035055467625899
Year: 2020 ... 217
2020	217	-1570.7684849999985	-7.2385644470046016
Year: 2021 ... 185
2021	185	-1466.3487219999997	-7.926209308108106
Year: 2022 ... 160
2022	160	-1103.6991919999998	-6.898119949999999


In [17]:
results_df = pd.DataFrame.from_records(results)
results_df.to_parquet(RESULTS_FILE)